In [32]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
#from tqdm import tqdm

In [33]:
ROOT_DIR = "data"
RANDOM_STATE = 110

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "shifted_train.csv"))
train_data

,Wip Line_Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE SPEED Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,...,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2,HEAD Standby Position X Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,target
0,IVI-OB6,Dam dispenser #1,AJX75334505,4F1XA938-1,100,16,14.9,8.4,14.7,1.04,...,835.5,458.0,428.0,243.7,114.612,19.9,7.0,127,1,Normal
1,IVI-OB6,Dam dispenser #1,AJX75334505,3KPM0016-2,70,10,21.3,4.9,21.3,1.49,...,835.5,458.0,156.0,427.9,270.000,19.6,7.0,185,1,Normal
2,IVI-OB6,Dam dispenser #2,AJX75334501,4E1X9167-1,85,16,14.7,8.5,14.7,1.61,...,305.0,499.8,1324.2,243.5,114.612,19.8,10.0,73,1,Normal
3,IVI-OB6,Dam dispenser #2,AJX75334501,3K1X0057-1,70,10,21.3,8.4,21.3,1.49,...,305.0,499.8,694.0,1324.2,270.000,19.9,12.0,268,1,Normal
4,IVI-OB6,Dam dispenser #1,AJX75334501,3HPM0007-1,70,10,9.7,4.9,9.6,0.67,...,835.5,458.0,156.0,427.9,270.000,19.7,8.0,121,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam dispenser #1,AJX75334501,3J1XF434-2,70,10,17.0,5.0,17.0,1.19,...,835.5,458.0,156.0,427.9,270.000,19.2,1.0,318,1,Normal
40502,IVI-OB6,Dam dispenser #2,AJX75334501,4E1XC796-1,100,16,14.9,8.5,14.7,1.04,...,304.8,499.8,1324.2,243.5,114.612,20.5,14.0,197,1,Normal
40503,IVI-OB6,Dam dispenser #1,AJX75334501,4C1XD438-1,100,16,14.2,8.2,14.3,0.99,...,835.5,458.0,428.0,243.7,85.000,19.7,1.0,27,1,Normal
40504,IVI-OB6,Dam dispenser #2,AJX75334501,3I1XA258-1,70,10,9.7,4.9,9.7,0.67,...,305.0,499.8,694.0,1324.2,270.000,20.1,13.0,117,1,Normal


In [34]:
len(train_data.columns)
train_data["HEAD Standby Position Y Collect Result_Dam"].unique()


array([66])

In [35]:
import pandas as pd

def drop_constant_columns(df):
    
    constant_columns = [col for col in df.columns if df[col].nunique() == 1]
    df = df.drop(columns=constant_columns)
    
    return df

train_data = drop_constant_columns(train_data)
train_data = train_data.drop(columns='CURE END POSITION X Collect Result_Fill2') # 1020 있는 컬럼

output_path = 'data/shifted_train_drop.csv'
train_data.to_csv(output_path, index=False)
len(train_data.columns)

90

In [37]:
numeric_train_data = train_data.select_dtypes(include=['number'])
print(len(numeric_train_data.columns))
numeric_train_data["target"] = train_data.iloc[:, -1]
print(len(numeric_train_data.columns))
print(numeric_train_data["target"].unique())
numeric_train_data['target'] = (numeric_train_data["target"]=='AbNormal')+0
print(numeric_train_data["target"].unique())

83
84
['Normal' 'AbNormal']
[0 1]


In [38]:
import lightgbm as lgb

x = numeric_train_data.drop(columns=['target'])
y = numeric_train_data['target']

lgb_X_train, lgb_X_test, lgb_Y_train, lgb_Y_test = train_test_split(x, y, train_size=0.8, random_state=110)
train_data = lgb.Dataset(lgb_X_train, label=lgb_Y_train)
test_data = lgb.Dataset(lgb_X_test, label=lgb_Y_test, reference=train_data)


In [39]:
params = {
    'objective': 'binary', 
    'metric': 'binary_logloss' # 성능 평가 지표 - model parameter / F1은 외부 계산
}

model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=100)

y_pred = model.predict(lgb_X_test)
thresold = 0.5 # 어떻게 할지 설정 - 함수 형태
y_pred_binary = [1 if x > thresold else 0 for x in y_pred]
f1 = f1_score(lgb_Y_test, y_pred_binary)
print(f'F1 Score: {f1:.4f}')


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1910, number of negative: 30494
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004739 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2407
[LightGBM] [Info] Number of data points in the train set: 32404, number of used features: 83
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.058943 -> initscore=-2.770427
[LightGBM] [Info] Start training from score -2.770427
F1 Score: 0.0907
